In [2]:
import tensorflow as tf

2023-04-24 13:27:09.671646: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /N/soft/rhel7/java/1.8.0_131/jre/lib/amd64/server/:/N/soft/rhel7/curl/intel/7.54.0/lib:/N/soft/rhel7/python/3.9.8/lib:/N/soft/rhel7/openmpi/gnu/4.0.4/lib:/N/soft/rhel7/libpng/1.2.57/lib:/N/soft/rhel7/gcc/9.3.0/lib64:/N/soft/rhel7/gcc/9.3.0/lib:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/ipp/lib/intel64:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/compiler/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/mkl/lib/intel64_lin:/N/soft/rhel7/intel/19.5/compilers_and_libraries_2019.5.281/linux/tbb/lib/intel64/gcc4.7:/N/soft/rhel7/intel/19.5/debugger_2019/iga/lib:/N/soft/rhel7/intel/19.5/d

In [3]:
import pandas as pd
from collections import Counter

In [4]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Model
import os
import random
import numpy as np
import pickle

In [5]:
from ReLERNN import *

In [133]:

def Count_SNP(window,df1,targets):
    lis2=[]
    targ={}

    for i in df1['Position']:
        if i!=None:
            valu=np.floor(int(i)/window)
            if valu in targ.keys():
                targ[valu]+=targets[int(valu)][0]
            else:
                targ[valu]=targets[int(valu)][0]
            lis2.append(valu)
            
 
    frequ = dict(Counter(lis2))
    
    for i in frequ.keys():
        targ[i]=targ[i]/frequ[i]
        
    return frequ,targ


def count_window(r):
    df1= pd.DataFrame(r.T,columns=['Position'])
    
    respectiveNormalizedTargets = [[t] for t in normalizeTargets()]
    targets = np.array(respectiveNormalizedTargets)
    
    window=10
    cal,targ= Count_SNP(window,df1,targets)
    idx=0
    lis=[]
    lis2=[]
    for i in cal:
        lis.append([idx,idx+window,cal[i]])
        lis2.append(targ[i])
        idx=idx+window
  

    tar_mean = np.mean(lis2)
    tar_sd = np.std(lis2)
    nTargets =lis2- tar_mean
    nTargets = np.divide(nTargets,tar_sd,out=np.zeros_like(nTargets),where=tar_sd!=0)

    return np.array(lis),np.array(nTargets)


def ALLELE_FREQ(window,df1,targets,haps):
    lis2=[]
    targ={}
    frequen={}
    for i in range(len(df1['Position'])):
        if i!=None:
            valu=np.floor(int(df1['Position'][i])/window)
            if valu in targ.keys():
                targ[valu]+=targets[i][0]
            else:
                targ[valu]=targets[i][0]
            lis2.append(valu)
            va= dict(Counter(haps[i]))

            frh = va[max(va, key=va.get)]
        
            sum_=sum(list(va.values()))
            frt = frh/sum_
            if valu in frequen.keys():
                frequen[valu]+=[frt]
            else:
                frequen[valu]=[frt]
            
 
    frequ = dict(Counter(lis2))

    for i in frequ.keys():
        targ[i]=targ[i]/frequ[i]
        
    return frequ,targ,frequen    
    


def SNP_WINDOW(w,r):
    df1= pd.DataFrame(r.T,columns=['Position'])
    
    respectiveNormalizedTargets = [[t] for t in normalizeTargets()]
    targets = np.array(respectiveNormalizedTargets)
    
    window=10
    cal,targ,frequen= ALLELE_FREQ(window,df1,targets,w)
    idx=0
    lis=[]
    lis_v=[]
    lis2=[]
    lis3_=[]
    lis3_1=[]
    for i in cal:
        lis.append([idx,idx+window*i,cal[i]])
        lis2.append(targ[i])
        C_v =dict(Counter(frequen[i]))
        
        data = list(C_v.items())
 
        lis_v.append(np.array(data).tolist()[0])
 
        
        idx=idx+window*i
  

    
    tar_mean = np.mean(lis2)
    tar_sd = np.std(lis2)
    nTargets =lis2- tar_mean
    nTargets = np.divide(nTargets,tar_sd,out=np.zeros_like(nTargets),where=tar_sd!=0)

    return np.array(lis),np.array(nTargets),np.array(lis_v)



def normalizeTargets(norm='zscore'):
    infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/',"info.p")
    nTargets_1 = pickle.load(open(infoFilename,"rb"))['rho']
    print(len(nTargets_1))

    if(norm == 'zscore'):
        tar_mean = np.mean(nTargets_1,axis=0)
        tar_sd = np.std(nTargets_1,axis=0)
        nTargets =nTargets_1- tar_mean
        nTargets = np.divide(nTargets,tar_sd,out=np.zeros_like(nTargets),where=tar_sd!=0)

    return nTargets_1

In [134]:


def __data_generation1(batchTreeIndices,val):

        haps = []
        pos = []
        for treeIndex in batchTreeIndices:
            print(treeIndex)
            Hfilepath = os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+val+'/val/',str(treeIndex) + "_haps.npy")
            Pfilepath = os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/'+val+'/val/',str(treeIndex) + "_pos.npy")
            print(Hfilepath)
            H = np.load(Hfilepath)
            P = np.load(Pfilepath)
            haps.append(H)
            pos.append(P)

        respectiveNormalizedTargets = [[t] for t in normalizeTargets()]
        targets = np.array(respectiveNormalizedTargets)
       
        return haps,pos


            
def __getitem__1(idx,val):
    X,x1= __data_generation1([idx],val)
    return X,x1


In [135]:
os.environ['PYTHONHASHSEED']=str(42)
random.seed(42)
np.random.seed(42)
#nProc = mp.cpu_count()
   

projectDir = '/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool'
trainDir = os.path.join(projectDir,"train")
valiDir = os.path.join(projectDir,"vali")
testDir = os.path.join(projectDir,"test")
networkDir = os.path.join(projectDir,"networks")


test_resultFile = os.path.join(networkDir,"testResults.p")
test_resultFig = os.path.join(networkDir,"testResults.pdf")
modelSave = os.path.join(networkDir,"model.json")
weightsSave = os.path.join(networkDir,"weights.h5")


maxSimS = 0
winFILE=os.path.join(networkDir,"windowSizes.txt")
with open(winFILE, "r") as fIN:
    for line in fIN:
        maxSimS=max([maxSimS, int(line.split()[5])])
maxSegSites = 0

for ds in [trainDir,valiDir,testDir]:
    DsInfoDir = pickle.load(open(os.path.join(ds,"info.p"),"rb"))
    segSitesInDs = max(DsInfoDir["segSites"])
    maxSegSites = max(maxSegSites,segSitesInDs)
maxSegSites = max(maxSegSites, maxSimS)




bds_train_params = {
    'treesDirectory':trainDir,
    'targetNormalization':"zscore",
    'batchSize': 64,
    'maxLen': 448,
    'frameWidth': 5,
    'shuffleInds':True,
    'sortInds':False,
    'center':False,
    'ancVal':-1,
    'padVal':0,
    'derVal':1,
    'realLinePos':True,
    'posPadVal':0,
    'seqD':20,
    'maf':0.05,
    'seed':42
            }


x_1,y = SequenceBatchGenerator(**bds_train_params).__getitem__(0)
x,pos= __getitem__1(0,'train')


vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0])

print(vale1)

#value = tf.convert_to_tensor(vale1,dtype=None)


#value.shape
xTrain = np.reshape(vale1, (vale1.shape[0], vale1.shape[1], 1))
sites= xTrain.shape
genotype_inputs = layers.Input(shape=(xTrain.shape[1],xTrain.shape[2]))

model1 = layers.Bidirectional(layers.GRU(84,return_sequences=False,input_shape=(1348,3)))(genotype_inputs)
model1 = layers.Dense(256)(model1)
model1 = layers.Dropout(0.35)(model1)
output = layers.Dense(1)(model1)

model1 = Model(inputs=genotype_inputs, outputs=[output])
model1.compile(optimizer='Adam', loss='mse')
model1.summary()





0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/0_haps.npy
13000
13000
[[0.95 1.  ]
 [0.95 1.  ]
 [0.95 1.  ]
 ...
 [0.95 1.  ]
 [0.75 2.  ]
 [0.95 1.  ]]
Model: "model_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_4 (InputLayer)        [(None, 2, 1)]            0         
                                                                 
 bidirectional_3 (Bidirectio  (None, 168)              43848     
 nal)                                                            
                                                                 
 dense_6 (Dense)             (None, 256)               43264     
                                                                 
 dropout_3 (Dropout)         (None, 256)               0         
                                                                 
 dense_7 (Dense)             (None, 1) 

In [136]:
vale1


array([[0.95, 1.  ],
       [0.95, 1.  ],
       [0.95, 1.  ],
       ...,
       [0.95, 1.  ],
       [0.75, 2.  ],
       [0.95, 1.  ]])

In [137]:
vali_x=[]
vali_y=[]
for i in range(1000):
    x,pos= __getitem__1(i,'vali')

    
    vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0])


 

    xvali =np.reshape(vale1, (vale1.shape[0], vale1.shape[1], 1))
    vali_x.append(xvali)
    vali_y.append(targets)

0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/0_haps.npy
13000
13000
1
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/1_haps.npy
13000
13000
2
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/2_haps.npy
13000
13000
3
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/3_haps.npy
13000
13000
4
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/4_haps.npy
13000
13000
5
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/5_haps.npy
13000
13000
6
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/6_haps.npy
13000
13000
7
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-mast

61
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/61_haps.npy
13000
13000
62
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/62_haps.npy
13000
13000
63
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/63_haps.npy
13000
13000
64
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/64_haps.npy
13000
13000
65
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/65_haps.npy
13000
13000
66
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/66_haps.npy
13000
13000
67
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/67_haps.npy
13000
13000
68
/N/u/samishr/Carbonate/Desktop/Recombination_landsca

122
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/122_haps.npy
13000
13000
123
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/123_haps.npy
13000
13000
124
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/124_haps.npy
13000
13000
125
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/125_haps.npy
13000
13000
126
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/126_haps.npy
13000
13000
127
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/127_haps.npy
13000
13000
128
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/128_haps.npy
13000
13000
129
/N/u/samishr/Carbonate/Desktop/Recomb

182
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/182_haps.npy
13000
13000
183
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/183_haps.npy
13000
13000
184
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/184_haps.npy
13000
13000
185
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/185_haps.npy
13000
13000
186
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/186_haps.npy
13000
13000
187
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/187_haps.npy
13000
13000
188
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/188_haps.npy
13000
13000
189
/N/u/samishr/Carbonate/Desktop/Recomb

242
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/242_haps.npy
13000
13000
243
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/243_haps.npy
13000
13000
244
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/244_haps.npy
13000
13000
245
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/245_haps.npy
13000
13000
246
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/246_haps.npy
13000
13000
247
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/247_haps.npy
13000
13000
248
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/248_haps.npy
13000
13000
249
/N/u/samishr/Carbonate/Desktop/Recomb

...

484
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/484_haps.npy
13000
13000
485
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/485_haps.npy
13000
13000
486
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/486_haps.npy
13000
13000
487
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/487_haps.npy
13000
13000
488
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/488_haps.npy
13000
13000
489
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/489_haps.npy
13000
13000
490
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/490_haps.npy
13000
13000
491
/N/u/samishr/Carbonate/Desktop/Recomb

544
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/544_haps.npy
13000
13000
545
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/545_haps.npy
13000
13000
546
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/546_haps.npy
13000
13000
547
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/547_haps.npy
13000
13000
548
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/548_haps.npy
13000
13000
549
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/549_haps.npy
13000
13000
550
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/550_haps.npy
13000
13000
551
/N/u/samishr/Carbonate/Desktop/Recomb

604
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/604_haps.npy
13000
13000
605
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/605_haps.npy
13000
13000
606
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/606_haps.npy
13000
13000
607
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/607_haps.npy
13000
13000
608
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/608_haps.npy
13000
13000
609
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/609_haps.npy
13000
13000
610
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/610_haps.npy
13000
13000
611
/N/u/samishr/Carbonate/Desktop/Recomb

664
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/664_haps.npy
13000
13000
665
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/665_haps.npy
13000
13000
666
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/666_haps.npy
13000
13000
667
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/667_haps.npy
13000
13000
668
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/668_haps.npy
13000
13000
669
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/669_haps.npy
13000
13000
670
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/vali/val/670_haps.npy
13000
13000
671
/N/u/samishr/Carbonate/Desktop/Recomb

KeyboardInterrupt: 

In [138]:
vali_x[0]

array([[[0.9 ],
        [1.  ]],

       [[0.95],
        [1.  ]],

       [[0.7 ],
        [1.  ]],

       ...,

       [[0.95],
        [2.  ]],

       [[0.85],
        [1.  ]],

       [[0.7 ],
        [1.  ]]])

In [139]:
callbacks_list = [
    EarlyStopping(
    monitor='val_loss',
    verbose=1,
    min_delta=0.01,
    patience=100),
    ModelCheckpoint(
    filepath='/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val',
    monitor='val_loss',
    save_best_only=True),
    TerminateOnNaN()]


In [140]:
for i in range(25):
    x,pos= __getitem__1(i,'train')


    
    
    vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0])
    

    value = tf.convert_to_tensor(vale1,dtype=None)
    
    xTrain =np.reshape(value, (value.shape[0], value.shape[1], 1))
    history = model1.fit(xTrain,targets,
    steps_per_epoch=10,
    epochs=1,
    callbacks=callbacks_list,
    validation_data=(vali_x[0],vali_y[0]),
    use_multiprocessing=True)
    #model1.fit(xTrain,targets,epochs=5)
    
    
model_json = model1.to_json()
with open('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/model', "w+") as json_file:
    json_file.write(model_json)



x,pos= __getitem__1(0,'test')
    
vale2,targets,vale1 = SNP_WINDOW(x[0],pos[0])

value = tf.convert_to_tensor(vale1,dtype=None)   

xTest = np.reshape(value, (value.shape[0], value.shape[1], 1))
predictions = model1.predict(xTest)

history.history['loss'] = np.array(history.history['loss'])
#history.history['val_loss'] = np.array(history.history['val_loss'])
history.history['predictions'] = np.array(predictions)
history.history['Y_test'] = np.array(targets)
history.history['name'] = 'ModelName'

pickle.dump(history.history, open( 'resultsFile', "wb" ))

0
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/0_haps.npy
13000
13000
 9/10 [==========================>...] - ETA: 0s - loss: 1.0358 

2023-04-24 14:45:09.423431: W tensorflow/python/util/util.cc:368] Sets are not currently considered sequences, but this may change in the future, so consider avoiding using them.


INFO:tensorflow:Assets written to: /N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/assets


INFO:tensorflow:Assets written to: /N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/assets


10/10 [==============================] - 17s 1s/step - loss: 1.0295 - val_loss: 1.0055
1
/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/val/1_haps.npy
13000
13000
 1/10 [==>...........................] - ETA: 0s - loss: 1.0545


KeyboardInterrupt



In [106]:
erro=[]
for i in range(1000):
    erro.append(np.linalg.norm((model1.predict(vali_x[i])-vali_y[i])*div))

In [109]:
de1= pd.DataFrame(erro)

In [97]:
div=  1/(4*ne)

In [66]:
div

array([272.11637365, 531.98701413, 275.87078574, ..., 316.59039428,
       306.42442032, 305.97317744])

In [110]:
de1

,0
0,0.000491
1,0.000484
2,0.000495
3,0.000495
4,0.000495
...,...
995,0.000495
996,0.000495
997,0.000495
998,0.000495


In [111]:
import seaborn as sns
import matplotlib.pyplot as plt
plt.clf()
sns.boxplot(data=de1)

plt.savefig("/N/u/samishr/Carbonate/Desktop/Recombination_landscape/Data_Processing/fig.png")

In [56]:
infoFilename= os.path.join('/N/u/samishr/Carbonate/Desktop/Recombination_landscape/ReLERNN-master/examples/example_output_pool/train/',"info.p")
ne = pickle.load(open(infoFilename,"rb"))['Ne']

In [51]:
nTargets_1

69982.4499438376

In [55]:
mu = pickle.load(open(infoFilename,"rb"))['mu']

In [53]:
nTargets_1

{'N': 20,
 'Ne': 69982.4499438376,
 'priorLowsRho': 0.0,
 'priorHighsRho': 1e-08,
 'priorLowsMu': 6.6e-09,
 'priorHighsMu': 1.3300000000000002e-08,
 'ChromosomeLength': 136200,
 'MspDemographics': None,
 'rho': array([3.74540119e-09, 9.50714306e-09, 7.31993942e-09, ...,
        8.67618866e-09, 3.66456646e-09, 4.32136995e-10]),
 'hotWin': None,
 'mu': array([1.31279282e-08, 6.71505904e-09, 1.29492661e-08, ...,
        1.12837416e-08, 1.16580924e-08, 1.16752855e-08]),
 'segSites': array([1966,  909, 1761, ..., 1459, 1690, 1989]),
 'winMasks': [[0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0513950073421439, [[0, 7000]], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.06607929515418502, [[0, 9000]], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.0, [], 0],
  [0.25697503671071953, [[0, 35000]], 0],
  [0.0, [], 0],
  [0.02936857562408223, [[0